In [1]:
import os 
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero
dir_ = '../../data/'
group_dir_ = os.path.join(dir_, 'groups/random')

In [2]:
file_name = 'normalized_to_rating_filter_track_5_user_100.csv'
all_user = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))
svd = pd.read_pickle(os.path.join(dir_, 'prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])

file_name = 'normalized_popularity_filter_track_5_user_100.pkl'
pop_count = pd.read_pickle(os.path.join(dir_, file_name))

num_user = len(svd['uid'].unique())
num_user

220

In [3]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8', '9', '10', '20']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)

In [4]:
pop_dict = {}
for i in all_user['tid'].unique():
    pop_dict[i] = len(all_user[all_user['tid']==i])/num_user

In [5]:
svd['count'] = 1
svd[:5]

,uid,tid,rating,count
0,0,3,2.544071,1
1,0,7,2.545822,1
2,0,18,2.152490,1
3,0,20,1.882396,1
4,0,31,2.410806,1


In [6]:
pop_count = pop_count.sort_values(by=['count'],  ascending=False)
pop_count[:10]

,tid,count,rating
166,166,3666,573.25
457,457,2042,162.00
6338,6338,1896,153.75
80,80,1835,412.25
1364,1364,1792,327.50
350,350,1667,359.75
7209,7209,1454,400.50
6282,6282,1397,17.75
13104,13104,1368,57.50
3761,3761,1340,354.50


In [7]:
# Linear
# size = int(len(pop_rating) // 5)
# pop_normalized = pd.DataFrame()
# for i in range(5):
#     if i == 0:
#         pop = pop_count[:size].copy()
#     elif i == 4:
#         pop = pop_count[size*4:].copy()
#     else:
#         pop = pop_count[size*i:size*(i+1)].copy()
#     pop['rating'] = (5 - i)
#     pop_normalized = pop_normalized.append(pop)
# pop_normalized = pop_normalized[['tid', 'rating']]

In [8]:
# min_max
max_rating = pop_count.iloc[0]['count']
pop_normalized = pop_count.copy()
pop_normalized = pop_normalized[['tid', 'count']]
pop_normalized['rating'] = pop_normalized['count']
pop_normalized['rating'] /= max_rating
pop_normalized['rating'] *= 5
pop_normalized[:10]

,tid,count,rating
166,166,3666,5.000000
457,457,2042,2.785052
6338,6338,1896,2.585925
80,80,1835,2.502728
1364,1364,1792,2.444081
350,350,1667,2.273595
7209,7209,1454,1.983088
6282,6282,1397,1.905346
13104,13104,1368,1.865794
3761,3761,1340,1.827605


In [9]:
lambdas = []
for i in range(11):
    lambdas.append(i/10)
lambdas

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [12]:
for lambda_ in tqdm(lambdas):
    top_n_size = 4
    top_n_lists_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = pop_normalized.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
        top_n_lists_g.append(top_n_lists)   
        
    threshold = 0.0
    avg_pops = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        pops = []
        for j in (range(len(groups))):
            group = groups[j]
            top_n_list = top_n_lists[j]
            for k in range(len(group)):
                for l in range(top_n_size):
                    tid = top_n_list[l]
                    pops.append(pop_dict[tid])                    
        avg_pop = sum(pops)/len(pops)
        avg_pops.append(avg_pop)
        
    print(avg_pops)

[0.48409090909090907, 0.48409090909090907, 0.48409090909090907, 0.48409090909090907, 0.48409090909091473, 0.4840909090909148, 0.48409090909091473, 0.48409090909091473, 0.4840909090909149, 0.4840909090909149]
[0.475606060606061, 0.4697159090909097, 0.47189393939394, 0.472111742424243, 0.47392676767677255, 0.4756598240469258, 0.4744107744107796, 0.4710227272727318, 0.4769628099173605, 0.4793388429752122]
[0.47920454545454594, 0.46994318181818256, 0.4703030303030309, 0.4711174242424248, 0.4732638888888935, 0.4707478005865149, 0.4744949494949548, 0.4722064393939438, 0.4710227272727323, 0.46983471074380667]
[0.48732954545454604, 0.469147727272728, 0.4642045454545459, 0.4727746212121218, 0.4782512626262669, 0.46983137829912475, 0.4704545454545505, 0.4665719696969737, 0.47122933884298007, 0.46508264462810367]
[0.4790340909090917, 0.4715340909090912, 0.46231060606060653, 0.4603219696969706, 0.46934974747475133, 0.45164956011730595, 0.44772727272727725, 0.45691287878788245, 0.4529442148760373, 